In [74]:
using CSV
using DataFrames
using Statistics

In [ ]:
results = CSV.read("latency_tests.csv", DataFrame, header = [
    "hostname",
    "job_id",
    "commit_id",
    "xrt_version",
    "repetition",
    "rank",
    "frame_size",
    "message_size",
    "iterations",
    "test_nfc",        
    "transmission_time",
    "failed_transmissions",
    "core_status",
    "fifo_status",
    "errors",
    "frames_received",
    "frames_with_errors"
])

results.latency = results.transmission_time ./ results.iterations
results.throughput = results.message_size ./ results.latency
results.throughput_gbit_s = results.throughput * 8 / 1e9

In [ ]:
function check(df)
    sort(
        combine(
            groupby(df, :hostname),
            nrow => :count,
            :failed_transmissions => sum => :failed_transmissions,
            :errors => sum => :byte_errors,
            :frames_with_errors => sum => :frame_errors,
        ),
        :hostname
    )
end

check(results)

In [ ]:
function aggregate_means(df)
    combine(
        groupby(df, [:frame_size, :message_size]),
        :latency => mean => :latency,
        :throughput => mean => :throughput,
        :throughput_gbit_s => mean => :throughput_gbit_s,
        nrow => :count,
        :failed_transmissions => sum => :failed_transmissions,
        :errors => sum => :byte_errors,
        :frames_with_errors => sum => :frame_errors,
    )
end

means = aggregate_means(results)

function tabulate(df, value)
    unstacked = unstack(
        df,
        :message_size,
        :frame_size,
        value
    )
    unstacked[:,vcat(
        ["message_size"],
        sort(setdiff(names(unstacked), ["message_size"]), by = fs -> parse(Int, fs))
    )]

end

display(tabulate(means, :count))
display(tabulate(means, :failed_transmissions))
display(tabulate(means, :byte_errors))
display(tabulate(means, :frame_errors))
display(tabulate(means, :latency))
display(tabulate(means, :throughput_gbit_s))